In [1]:
import json
import requests
import time

## Load dataset

In [2]:
with open("../train-v2.0.json", 'r') as f:
    data = json.load(f)

In [3]:
def get_qa(topic, data):
    q = []
    a = []
    for d in data['data']:
        if d['title']==topic:
            for paragraph in d['paragraphs']:
                for qa in paragraph['qas']:
                    if not qa['is_impossible']:
                        q.append(qa['question'])
                        a.append(qa['answers'][0]['text'])
            return q,a

questions, answers = get_qa(topic='Premier_League', data=data)
print("Number of available questions: {}".format(len(questions)))

Number of available questions: 357


## Run app

In [4]:
%%bash
docker build ../ -t qamodel 
docker run -d -p 8000:8000 qamodel

Sending build context to Docker daemon  267.6MB
Step 1/6 : FROM python:3.8.12-slim-buster
 ---> d78a351dcf4a
Step 2/6 : RUN apt-get update &&    apt-get install -y --no-install-recommends wget &&    rm -rf /var/lib/apt/lists/* &&    pip install --no-cache-dir transformers[torch] uvicorn fastapi
 ---> Using cache
 ---> ad8f6df7006a
Step 3/6 : COPY download_model.sh .
 ---> Using cache
 ---> ab147f33d0d9
Step 4/6 : RUN bash download_model.sh
 ---> Using cache
 ---> d9dca610aa69
Step 5/6 : COPY app/ app/
 ---> Using cache
 ---> 3872e51fecd4
Step 6/6 : CMD ["python", "app/main.py"]
 ---> Using cache
 ---> 899286aece5b
Successfully built 899286aece5b
Successfully tagged qamodel:latest
d952b8810b05de6b61139efb9d3646e65d7ac65e5a8c4fb8ee0db71fd7098bfc


In [5]:
time.sleep(10)

In [6]:
json_data = {
  'questions':questions,
  'answers':answers,
}

response = requests.post(
  'http://0.0.0.0:8000/set_context',
  json=json_data
)

response.json()

{'message': 'Search context set'}

In [7]:
new_questions = [
    'How many teams compete in the Premier League ?',
    'When does the Premier League starts and finishes ?',
    'Who has the highest number of goals in the Premier League ?',
]

json_data = {
  'questions':new_questions,
}

response = requests.post(
  'http://0.0.0.0:8000/get_answer',
  json=json_data
)

for d in response.json():
  print('\n'.join(["{} : {}".format(k, v) for k,v in d.items()])+'\n')

orig_q : How many teams compete in the Premier League ?
best_q : How many clubs are currently in the Premier League?
best_a : 20

orig_q : When does the Premier League starts and finishes ?
best_q : When does the Premier League have its playing season?
best_a : During the course of a season (from August to May)

orig_q : Who has the highest number of goals in the Premier League ?
best_q : Who has the record for most goals in the Premier League?
best_a : Newcastle United striker Alan Shearer holds the record for most Premier League goals with 260

